In [1]:
%load_ext autoreload
%autoreload 2

import matplotlib
import matplotlib.pyplot as plt
#matplotlib.use('Agg')
%matplotlib tk
%autosave 180

import sys
sys.path.append('/home/cat/code/gerbil/utils/')

#
import numpy as np

#
#from visualize import visualize
from track import track as Track


Autosaving every 180 seconds


In [2]:
######################################################## 
################ GENERATE FIXED TRACK ##################
########################################################

# STEP 1: Convert both .slp files (before and after Id-swtch) to .npy files and fix them
# RUN: this cell 2 x

# 
fnames_slp = [
    '/media/cat/4TBSSD/dan/cohort2/P21_P28/2020_07_22_05_54_14_824412_compressed_Night.slp',
    '/media/cat/4TBSSD/dan/cohort2/P21_P28/2020_07_22_05_54_14_824412_compressed_defished_shrink_cropped.mp4.predictions.slp'
]

for fname_slp in fnames_slp:
    track = Track.Track  (fname_slp)
    track.track_type = 'features'

    ###### parameters for computing body centroid #######
    track.use_dynamic_centroid = True   # True: alg. serches for the first non-nan value in this body order [2,3,1,0,4,5]
                                         # - advantage: much more robust to lost features
                                         # False: we fix the centroid to a specific body part
                                         # - advantage less jitter for some applications
    track.centroid_body_id = [2]         # if centroid flag is False; we use this body part instead

    ##### run track fixer #######
    track.fix_all_tracks()

    ##### join spatially close but temporally distant chunks #####
    if True:
        #
        track.memory_interpolate_tracks_spine()

    ##### save the fixed spines will overwrite the previous/defatul spine values####
    track.save_centroid()

    #
    print ("Done...")

track_names: 6
node_names: 6
edge_names: 5
edge_inds: 5
tracks: (28802, 6, 2, 6)
track_occupancy: (6, 28802)
point_scores: (28802, 6, 6)
instance_scores: (28802, 6)
tracking_scores: (28802, 6)
labels_path: None
video_path: /scratch/cm5635/cohort2/2020_07_22_05_54_14_824412_compressed_defished_shrink_cropped.mp4
video_ind: 0
provenance: {}
Saved as /media/cat/4TBSSD/dan/cohort2/P21_P28/2020_07_22_05_54_14_824412_compressed_Night.h5
Done...
track_names: 6
node_names: 6
edge_names: 5
edge_inds: 5
tracks: (28802, 6, 2, 6)
track_occupancy: (6, 28802)
point_scores: (28802, 6, 6)
instance_scores: (28802, 6)
tracking_scores: (28802, 6)
labels_path: None
video_path: /mnt/b3a68699-495d-4ebb-9ab1-ac74f11c68c5/id_switch/cohort2/night/2020_07_22_05_54_14_824412_compressed_defished_shrink_cropped.mp4
video_ind: 0
provenance: {"model_paths": ["/mnt/b3a68699-495d-4ebb-9ab1-ac74f11c68c5/id_switch/cohort2/night/models/gerbils.multiclass_bottomup/training_config.json"], "predictor": "BottomUpMultiClassPr

In [3]:
####################################################
########## MAKE PLOTS OF SPINE CENTRES #############
####################################################

# OPTIONAL VISUALIZE TRACKS
def plot_track(track, animal_id):
    
    #
    plt.plot(track[:,animal_id,0],
             track[:,animal_id,1],
             #label=str(k),

            )

    plt.xlim(0,1000)
    plt.ylim(0,800)

#
animal_id = 0
    
#    
track = np.load(fnames_slp[0].replace('.slp','_spine.npy'))
plot_track(track, animal_id)

#    
track = np.load(fnames_slp[1].replace('.slp','_spine.npy'))
plot_track(track, animal_id)


In [11]:
###########################################################
###########################################################
###########################################################
def check_switch(tracks, track, animal_id, threshold=10):
    
    #
    tracks = np.delete(tracks,animal_id, axis=0)

    #
    dists = np.linalg.norm(tracks-track, axis=1)
    if np.nanmin(dists)<threshold:
        return True
    
    return False
    
    
#
def find_bouts(track,animal_id):
    from tqdm import trange
    bouts = []
    track1 = track[:,animal_id]
    
    starts = []
    ends = []
    switches=0
    if np.isnan(track1[0,0])==False:
        starts.append(0)
    
    #
    for k in trange(1,track1.shape[0]-1,1):
        if np.isnan(track1[k,0]) and np.isnan(track1[k+1,0])==False:
            starts.append(k+1)
        #
        elif np.isnan(track1[k,0])==False and np.isnan(track1[k+1,0])==True:
            ends.append(k)
            
            # also check if the next frame is a switch
            res = check_switch(track[k+1], track1[k], animal_id)
            
            if res:
                switches+=1
    
    #
    max_len = min(len(starts),len(ends))

    starts = starts[:max_len]
    ends = ends[:max_len]
    #print ("starts: ", len(starts))
    #print ('ends: ', len(ends))
    
    
    bouts = np.vstack((starts,ends)).T
    print ("bouts: ", bouts)
    
    return bouts, switches



def get_n_frames(track, animal_id):
    
    # get # of frames with animal detected
    #print (track.shape)
    s = track[:,animal_id].sum(1)
    
    idx = np.where(np.isnan(s)==False)[0]
    #print (s.shape, idx.shape)
    
    n_frames = idx.shape[0]
    
    return n_frames
    
def get_ave_features(full_track, animal_id):
    # get average # of detected frames
    
    #
    full_track = full_track[:,animal_id].sum(2)
    
    # find non-nan values 
    idx2 = np.where(np.isnan(full_track)==False)

    # make an array
    idx2 = np.array(idx2)[0]
    #print (idx2)
    
    # for each frame count the # of non-nan features
    n_unique, counts = np.unique(idx2, return_counts=True)
 
    # make empty list to start
    n_feats=np.zeros(full_track.shape[0])
 
    return np.mean(counts)


##########################################
##########################################
##########################################
animal_ids = np.arange(6)
plt.figure()

# 
for ctr, animal_id in enumerate(animal_ids):
    print ("processing animal: ", animal_id)
    #
    track = np.load(fnames_slp[0].replace('.slp','_spine.npy'))
    bouts1,switches1 = find_bouts(track, animal_id)

    track = np.load(fnames_slp[1].replace('.slp','_spine.npy'))
    bouts2,switches2 = find_bouts(track, animal_id)
    
    #
    spine_track = np.load(fnames_slp[0].replace('.slp','_spine.npy'))
    n_frames1 = get_n_frames(spine_track, animal_id)
    spine_track = np.load(fnames_slp[1].replace('.slp','_spine.npy'))
    n_frames2 = get_n_frames(spine_track, animal_id)
    
    #
    full_track = np.load(fnames_slp[0].replace('.slp','.npy'))
    ave_feats1 = get_ave_features(full_track, animal_id)
    full_track = np.load(fnames_slp[1].replace('.slp','.npy'))
    ave_feats2 = get_ave_features(full_track, animal_id)
    #print ("averages: ", ave_feats1, ave_feats2)

    # 
    ax=plt.subplot(2,3,1)
    
    if ctr==0:
        plt.bar(ctr, bouts1.shape[0], 0.4, color='black', label='original')
        plt.bar(ctr+0.4, bouts2.shape[0], 0.4, color='blue', label='350fixed')
    else:
        plt.bar(ctr, bouts1.shape[0], 0.4, color='black')
        plt.bar(ctr+0.4, bouts2.shape[0], 0.4, color='blue')
    plt.legend()
    plt.ylabel("# of segments")
    
    #
    ax2=plt.subplot(2,3,2)
    max_ = 1800
    plt.title("350fixed - original")
    #plt.semilogy()
    diffs1 = bouts1[:,1]-bouts1[:,0]
    y1= np.histogram(diffs1, bins=np.arange(0,max_,10))
    
    #
    diffs2 = bouts2[:,1]-bouts2[:,0]
    y2= np.histogram(diffs2, bins=np.arange(0,max_,10))
    
    plt.plot(y1[1][:-1],y2[0]-y1[0], label=str(ctr))
    plt.plot([0,max_],[0,0],'--',c='grey')
    plt.xlabel("duration of bout")
    plt.ylabel("# of bouts")
    plt.legend()
    
    #
    ax3=plt.subplot(2,3,3)
    plt.bar(ctr, diffs1.sum()/track.shape[0], 0.4, color='black')
    plt.bar(ctr+0.4, diffs2.sum()/track.shape[0], 0.4, color='blue')  
    plt.ylabel("total duration of tracked bouts ")
    plt.xlabel("animal id")
    
    ax3=plt.subplot(2,3,4)
    plt.bar(ctr, switches1, 0.4, color='black')
    plt.bar(ctr+0.4, switches2, 0.4, color='blue')  
    plt.ylabel("Likely ID switches")
    plt.xlabel("animal id")
    
    ax4=plt.subplot(2,3,5)
    plt.bar(ctr, n_frames1, 0.4, color='black')
    plt.bar(ctr+0.4, n_frames2, 0.4, color='blue')  
    plt.ylabel("n_frames detected")
    plt.xlabel("animal id")
    
    ax4=plt.subplot(2,3,6)
    plt.bar(ctr, ave_feats1, 0.4, color='black')
    plt.bar(ctr+0.4, ave_feats2, 0.4, color='blue')  
    plt.ylabel("ave # of features/frame")
    plt.xlabel("animal id")
    
#
plt.show()



processing animal:  0


100%|██████████| 28800/28800 [00:00<00:00, 58245.46it/s]


bouts:  [[    0   158]
 [  165   175]
 [  188   220]
 [  240   250]
 [  258  2538]
 [ 3191  3195]
 [ 3244  3274]
 [ 3287  3304]
 [ 3326  3331]
 [ 3758  3772]
 [ 3819  8818]
 [ 8823  8835]
 [ 8848  8881]
 [ 8883 10249]
 [10871 10987]
 [11204 11269]
 [11293 11455]
 [11483 11534]
 [11625 11630]
 [11649 11653]
 [11676 11734]
 [11738 11808]
 [11816 11820]
 [11865 11948]
 [12016 12022]
 [12051 12055]
 [12078 12122]
 [12129 12144]
 [12197 12201]
 [12219 12223]
 [12320 12325]
 [12337 12350]
 [12386 12509]
 [12517 12529]
 [12572 12610]
 [12618 12664]
 [12668 12716]
 [12723 12735]
 [12786 12791]
 [12834 12856]
 [12878 12915]
 [12923 12936]
 [12956 12971]
 [12987 13023]
 [13034 13105]
 [13163 13196]
 [13204 13231]
 [13250 13267]
 [13287 13294]
 [13302 13317]
 [13338 13359]
 [13407 13411]
 [13420 13426]
 [13440 13465]
 [13504 13555]
 [13560 13595]
 [13612 13620]
 [13636 13646]
 [13653 13657]
 [13691 13721]
 [13726 13746]
 [13769 13810]
 [13812 14690]
 [14697 15909]
 [15938 16422]
 [16462 16504]
 [

100%|██████████| 28800/28800 [00:00<00:00, 61406.24it/s]


bouts:  [[ 8831  8824]
 [21831 21823]
 [23621 23609]
 [23689 23671]
 [23725 23699]
 [23752 23729]
 [23791 23756]
 [24083 24075]
 [24094 24089]
 [24109 24099]
 [26439 25749]
 [26921 26919]
 [26985 26973]
 [26993 26989]
 [27227 27219]
 [27346 27344]
 [27582 27574]
 [28539 28248]
 [28640 28613]
 [28680 28678]]
processing animal:  1


...

bouts:  [[    0    31]
 [   34   143]
 [  154   196]
 [  206   213]
 [  228   265]
 [  269   695]
 [  707   749]
 [  751   758]
 [  760  1436]
 [ 1464  2288]
 [ 2291  2299]
 [ 2322  2364]
 [ 2367  2371]
 [ 2384  2391]
 [ 2400  2447]
 [ 2449  2454]
 [ 2465  2471]
 [ 2479  2519]
 [ 2532  2537]
 [ 2539  2555]
 [ 2583  2690]
 [ 2729  2737]
 [ 2749  3249]
 [ 3265  3293]
 [ 3295  3365]
 [ 3382  3820]
 [ 3831  3941]
 [ 3962  5385]
 [ 5396  5407]
 [ 6026  6745]
 [ 6756  6767]
 [ 6920  6924]
 [ 7456  7828]
 [ 7852  8704]
 [ 8717  8790]
 [ 8816  8847]
 [13239 14104]
 [14128 14139]
 [14338 14373]
 [14382 14425]
 [14636 14648]
 [14719 14724]
 [15324 15648]
 [16001 16005]
 [16240 16542]
 [16563 16618]
 [17344 17379]
 [17407 17488]
 [17499 17504]
 [17506 17546]
 [17607 17611]
 [23025 23057]
 [23497 23599]
 [23615 23712]
 [23727 23787]
 [23791 23798]
 [23833 23868]
 [23896 23943]
 [23947 23951]
 [23953 24012]
 [24054 24074]
 [24496 24538]
 [24556 24562]
 [24999 25083]
 [25154 25159]
 [25188 25224]
 [

100%|██████████| 28800/28800 [00:00<00:00, 62747.53it/s]


bouts:  [[10876 17501]
 [17508 17574]
 [18940 18973]
 [19235 19249]
 [19266 19276]
 [19416 19420]
 [19654 19680]
 [20373 20380]
 [22263 22271]
 [22276 22491]
 [24655 24659]
 [24661 24667]
 [25022 25027]
 [25663 25674]
 [26205 26266]
 [26366 26371]
 [26445 26496]
 [26683 26756]
 [26850 27066]
 [27084 27097]
 [27190 27197]
 [27219 27312]
 [27329 27371]
 [27395 27441]
 [27505 27529]
 [27609 27672]
 [27797 27811]
 [27924 27978]
 [27995 28012]
 [28097 28109]
 [28138 28165]
 [28179 28229]
 [28258 28409]
 [28414 28422]
 [28441 28471]
 [28542 28564]
 [28575 28579]
 [28581 28605]
 [28616 28637]
 [28682 28693]
 [28703 28769]
 [28779 28797]]


100%|██████████| 28800/28800 [00:00<00:00, 62837.98it/s]


bouts:  [[ 2270  2285]
 [ 2292  2315]
 [10843 10880]
 [23797 23811]
 [23941 25027]
 [26471 26490]
 [26954 26959]
 [26978 26989]
 [27009 27048]
 [27051 27097]
 [27106 27197]
 [27220 27228]
 [27294 27372]
 [27440 27539]
 [27549 27554]
 [27595 27727]
 [27757 27767]
 [27784 27839]
 [27843 28088]
 [28096 28132]
 [28142 28422]
 [28424 28428]
 [28435 28561]
 [28568 28605]
 [28616 28635]
 [28682 28784]]
